In [97]:
import pandas as pd
from bs4 import BeautifulSoup
import csv
from collections import defaultdict
import numpy as np
import math
import io
import os
import plotly.graph_objects as go
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from lxml import html
import index_utils
import heapq
ps = PorterStemmer()
import json

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/salwaali/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/salwaali/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 1.3 Parse downloaded pages


In [ ]:
# #### In the following lines of code we will extract all the valuable and needed information from the saved html files.  We need to extract the title, intro, plot of the movie as well as some ifobox information like director, producer, writter, stars starring in the film, music, release date, running time, country, language and budget.
# #### In the following file you could find the parsing of the movies in movie2.html list. To parse the other files you can change the path directory.

for i in range(10000):
    #reading all the articles, the path changed according to the DS 
    movie = BeautifulSoup (open(r"/Users/salwaali/Desktop/FINAL_ADM_HW3/movies/artical_%d.html" %i, encoding="utf8"), features="html")
    #movie = BeautifulSoup (open(r"/Users/salwaali/Desktop/FINAL_ADM_HW3/movies_2/mov2/artical_%d.html" %i, encoding="utf8"), features="html")
    #movie = BeautifulSoup (open(r"/Users/salwaali/Desktop/FINAL_ADM_HW3/movies_3/movies_3/article_%d.html" %i, encoding="utf8"), features="html")


    # =======================================================================================
    # We have created a function so as to make the information more clear and remove the unicode codes that exist
    def beau(word):
        return(re.sub('[{1}[0-9]*]{1}', '',  word.strip('\n').replace('\xa0', ' ')).split('\n'))
    #=================================================================
    #finding the title of the movie
    for t in movie.select('title'):
        title =(t.text).split(" - ")[0]
    title = "".join(beau(title))
    # ================================================================
    #finding the intro of the movie
    intro = []
    flagi = 0
    # we go through the different layers of html and we search for the specific 
    # paragraph that corresponds to the intro
    for div in movie.findAll("div",{"class":"mw-parser-output"}):
        for p in div.children:
            if p.name == "p":
                intro.append(p.text)
            if p.name == "h2":
                break
    # since we didnt find an intro from the previous code, we will assign the value NaN to it 
    if intro == []:
        intro = float("NaN")
        flagi = 1
    # if the search for the intro was successful we will apply the function to erase any imperfection
    if flagi == 0:
        intro = "".join(beau("".join(intro)))
    # =================================================================
    #finding the plot of the movie
    par = []
    itsin = 0
    flagpp=0
    # we go through the different layers of html and we search for the specific 
    # paragraph that corresponds to the plot(sometimes denoted as plot summary, Plot Summary, Story, Summary etc)
    for section in movie.find_all('h2'):
        for t in section.find_all("span",{'id':'Plot'}):
            itsin = 1
            continue
        for t in section.find_all("span",{'id':'Plot_summary'}): 
            itsin = 1
            continue
        for t in section.find_all("span",{'id':'Plot_Summary'}): 
            itsin = 1
            continue
        for t in section.find_all("span",{'id':'Story'}): 
            itsin = 1
            continue
        for t in section.find_all("span",{'id':'Summary'}): 
            itsin = 1
            continue
        for t in section.find_all("span",{'id':'Premise'}): 
            itsin = 1
            continue
        for t in section.find_all("span",{'id':'Synopsis'}): 
            itsin = 1
            continue
        for t in section.find_all("span",{'id':'Storyline'}): 
            itsin = 1
            continue
        for t in section.find_all("span",{'id':'Storylines'}): 
            itsin = 1
            continue
        # if we find the tag that has attribute "Plot" we follow that and it will lead us to the
        # paragraphs containing the plot
        if itsin == 1:
            for p in section.find_next_siblings():
                if p.name == 'h2':
                    flagpp=1
                    break
                if p.name == "p":
                    par.append(p.text)
                if p.name == "ol":
                    for l in p.find_all("li"):
                        par.append(l.text)
            if flagpp == 1:
                break
    # if we dont find the plot in the previouw way , we try with the h3 way as follows
    if flagpp == 0 and par == []:
        for section in movie.find_all('h3'):
            for t in section.find_all("span",{'id':'Plot'}):
                itsin = 1
                continue
            for p in section.find_next_siblings():
                    if p.name == 'h2':
                        flagpp=1
                        break
                    if p.name == "p":
                        par.append(p.text)
                    if p.name == "ol":
                        for l in p.find_all("li"):
                            par.append(l.text)
            if flagpp == 1:
                break
    # since we didnt find an plot from the previous code, we will assign the value NaN to it 
    if par == []:
        par = float("NaN")
    else:
        par = "".join(beau("".join(par)))
    #===================================================================
    # I_N_F_O_B_O_X
    d = defaultdict(list)
    for div in movie.findAll("div",{"class":"mw-body"}):
        for div1 in div.findAll('div',{"class":"mw-body-content"}):
            for div2 in div1.findAll('div',{"class":"mw-content-ltr"}):
                for table in div2.findAll('table',{"class":"infobox vevent"}):
                    for tr in table.findAll("tr"):
                        for th1 in tr.findAll("th"):
                            for td1 in tr.findAll("td"):
                                for j in range(len(td1.contents)):
                                    attr = []
    # we are creating a dictionary. the keys are the column that correctonds to 
    # Directed by, Produced by etc and the values are the corresponding answers like director, producer etc
                                    try:
                                        attr.append(td1.contents[j].text)
                                        d[th1.text].append(attr)
    # because sometimes the text nodes contain links we use try and except. we use the except part
    # in case we cannot access the information with the try code
                                    except AttributeError:
                                        attr.append(td1.contents[j])
                                        d[th1.text].append(attr)

    # =============================================================================
    # Get only the information that we need
    final_info=[]
    keys = ["Directed by", "Produced by", "Written by", 
       "Starring", "Music by", "Release date", "Running time", 
       "Country", "Language", "Budget"]
    for ele in keys:
        info = ""
        if ele in d.keys():
            if len(ele)>0:
                for i_ in range(0,len(d[ele]),2):
                    info = info  + " " + "".join(beau(d[ele][i_][0]))
            else:
                info = float("NaN")
        final_info.append(info.strip())
    # ============================================================================
    # Correct the unicode issues
    final_info[3] = " ".join(re.findall('[A-Z][^A-Z]*', final_info[3])) 
    # ===========================================================================
    # In the written by place on the infobox there are the information about the original 
    # screenplay and screenplay.. We just summed up all the names
    if "Original screenplay" or "Screenplay" in final_info[2]:
        final_info[2] = final_info[2].replace("Original screenplay" and "Screenplay", "")
    # =====================================================================================
    # All the empty elements are taking the value NaN
    for k in range(len(final_info)):
        if final_info[k] == "" or final_info[k] == ['N/A'] :
            final_info[k] = float("NaN")
    # T_S_V
    data=['Title', 'Intro', 'Plot', 'Directed by','Produced by','Written by','Starring','Music by',
          'Release date','Running time','Country','Language','Budget']      
    with open(r"/Users/salwaali/Desktop/FINAL_ADM_HW3/Final_tsv/T_S_V_M_%d.tsv" %i, 'wt', encoding = 'utf8') as f:
        tsv_file = csv.writer(f, delimiter='\t')
        tsv_file.writerow(data)
        tsv_file.writerow([title, intro, par, final_info[0], final_info[1], final_info[2], final_info[3], final_info[4],final_info[5], final_info[6], final_info[7], final_info[8], final_info[9]])

# Combining the tsv files to one DF

In [56]:
movies1 = BeautifulSoup (open(r"/Users/salwaali/Desktop/FINAL_ADM_HW3/DS/movies1.html"), features="html")
movies_links1 = []
for url in movies1.find_all("a"):
    movies_links1.append(url.get("href"))
#====================
movieDatabase1 = pd.read_csv(r"/Users/salwaali/Desktop/FINAL_ADM_HW3/Final_tsv/T_S_V_M_0.tsv", sep='\t', header=0)
movieDatabase1 = pd.read_csv(r"/Users/salwaali/Desktop/FINAL_ADM_HW3/Final_tsv/T_S_V_M_0.tsv", sep='\t', header=0)
for i in range(1,10000):
    movieDatabase1 = movieDatabase1.append(pd.read_csv(r"/Users/salwaali/Desktop/FINAL_ADM_HW3/Final_tsv/T_S_V_M_%d.tsv"  %i, sep='\t'), ignore_index=True)
movieDatabase1["URL"] = movies_links1
#=================
movies2 = BeautifulSoup (open(r"/Users/salwaali/Desktop/FINAL_ADM_HW3/DS/movies2.html"), features="html")
movies_links2 = []
for url in movies2.find_all("a"):
    movies_links2.append(url.get("href"))
#========================
for i in range(1,10000):
    movieDatabase2 = movieDatabase2.append(pd.read_csv(r"/Users/salwaali/Desktop/FINAL_ADM_HW3/Final_tsv_2/T_S_V_M_%d.tsv"  %i, sep='\t'), ignore_index=True)

movieDatabase2["URL"] = movies_links2
movieDatabase2.head(3)
#=====================
movies3 = BeautifulSoup (open(r"/Users/salwaali/Desktop/FINAL_ADM_HW3/DS/movies3.html"), features="html")
movies_links3 = []
for url in movies3.find_all("a"):
    movies_links3.append(url.get("href"))
movies_links3[0:3]
#=======================
movieDatabase3 = pd.read_csv(r"/Users/salwaali/Desktop/FINAL_ADM_HW3/Final_tsv_3/T_S_V_M_0.tsv", sep='\t', header=0)
for i in range(1,10000):
    movieDatabase3 = movieDatabase3.append(pd.read_csv(r"/Users/salwaali/Desktop/FINAL_ADM_HW3/Final_tsv_3/T_S_V_M_%d.tsv"  %i, sep='\t'), ignore_index=True)

movieDatabase3["URL"] = movies_links3
movieDatabase3.head(3)

In [25]:
movieDatabase=pd.concat([movieDatabase1,movieDatabase2,movieDatabase3], ignore_index=True)

In [26]:
movieDatabase

,Title,Intro,Plot,Directed by,Produced by,Written by,Starring,Music by,Release date,Running time,Country,Language,Budget,URL
0,Love by the Light of the Moon,Love by the Light of the Moon is a 1901 film b...,"The moon, painted with a smiling face hangs ov...",Edwin S. Porter,NaN,NaN,NaN,NaN,1901 (1901),NaN,United States,Silent,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...
1,The Martyred Presidents,The Martyred Presidents is a 1901 American fil...,"The film, just over a minute long, is composed...",Edwin S. Porter,NaN,NaN,See below,NaN,1901,1 minutes,United States,English,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...
2,"Terrible Teddy, the Grizzly King","Terrible Teddy, the Grizzly King is a 1901 Ame...",Lasting just 61 seconds and consisting of two ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_..."
3,Jack and the Beanstalk (1902 film),Jack and the Beanstalk is a 1902 American shor...,In this earliest known adaptation of the class...,George S. Fleming Edwin S. Porter,Edison Studios,NaN,See below,NaN,1902,10 minutes,United States,English,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...
4,Alice in Wonderland (1903 film),Alice in Wonderland is a 1903 British silent f...,"Alice follows a large white rabbit down a ""Rab...",Cecil Hepworth Percy Stow,Cecil M. Hepworth Herman Casler (exec. produce...,Lewis Carroll,May Clark Cecil M. Hepworth Mrs. Cecil ...,NaN,17 October 1903 (1903-10-17),approx. 8:19 minutes,United Kingdom,Silent,NaN,https://en.wikipedia.org/wiki/Alice_in_Wonderl...
5,The Great Train Robbery (1903 film),The Great Train Robbery is a 1903 American sil...,The film opens with two bandits breaking into ...,Edwin S. Porter,Edwin S. Porter,NaN,Alfred C. Abadie Broncho Billy Anderson ...,NaN,"December 1, 1903 (1903-12-01)",12 minutes (at 18 frame/s),United States,Silent English,$,https://en.wikipedia.org/wiki/The_Great_Train_...
6,The Suburbanite,The Suburbanite is a 1904 American short comed...,The film is about a family who move to the sub...,Wallace McCutcheon,NaN,Frank Marion,John Troiano,NaN,November 1904,NaN,United States,Silent,NaN,https://en.wikipedia.org/wiki/The_Suburbanite
7,The Little Train Robbery,The Little Train Robbery is a parodic sequel t...,The opening scene shows the interior of the ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://en.wikipedia.org/wiki/The_Little_Train...
8,The Night Before Christmas (1905 film),The Night Before Christmas is a 1905 American ...,Scenes are introduced using lines of the poem....,Edwin S. Porter,NaN,Clement Clarke Moore,NaN,NaN,"December 16, 1905 (1905-12-16)",9:00,United States,English,NaN,https://en.wikipedia.org/wiki/The_Night_Before...
9,Dream of a Rarebit Fiend (1906 film),Dream of a Rarebit Fiend is a 1906 American si...,The Rarebit Fiend gorges on Welsh rarebit at a...,Wallace McCutcheon Edwin S. Porter,NaN,Winsor McCay,Jack Brawn,NaN,1906 (1906),6 minutes,United States,Silent,NaN,https://en.wikipedia.org/wiki/Dream_of_a_Rareb...


# Droping Null

In [27]:
truth = []
for i in range(len(movieDatabase)):
    al = (movieDatabase.Plot[i] != movieDatabase.Plot[i] and 
          movieDatabase["Directed by"][i] != movieDatabase["Directed by"][i] and
         movieDatabase["Produced by"][i] != movieDatabase["Produced by"][i] and
         movieDatabase["Written by"][i] != movieDatabase["Written by"][i] and
         movieDatabase["Starring"][i] != movieDatabase["Starring"][i])
    truth.append(not(al))
movieDatabase = movieDatabase[truth]

# Save tha DS for further use 

In [29]:
new = movieDatabase[['Title', 'Intro', 'URL']].copy()
new.to_csv("searchenginedata1.csv", index=False )

In [30]:
movieDatabase.reset_index(inplace = True)
movieDatabase.drop(columns = ["index"], inplace = True)

/Users/salwaali/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



# Clean our DS for our search engins 

In [54]:
def cleanQ(query):
    query = query.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(query)
    stemmer=[ps.stem(i) for i in tokens]
    filtered_Q = [w for w in stemmer if not w in stopwords.words('english')]
    return filtered_Q

In [55]:
tmp_intro = []
tmp_title = []
tmp_plot = []
tmp_director = []
tmp_producer = []
tmp_writer = []
tmp_starring = []
tmp_music = []
tmp_date = []
tmp_country = []
tmp_language = []
tmp_time = []
for i in range(len(movieDatabase)):
    if movieDatabase["Intro"][i] == movieDatabase["Intro"][i]:
            tmp_intro.append(cleanQ(movieDatabase["Intro"][i]))
    else:
        tmp_intro.append("")
    if movieDatabase["Plot"][i] == movieDatabase["Plot"][i]:
            tmp_plot.append(cleanQ(movieDatabase["Plot"][i]))
    else:
        tmp_plot.append("")
    if movieDatabase["Title"][i] == movieDatabase["Title"][i]:
        tmp_title.append(cleanQ(movieDatabase["Title"][i]))
    else:
        tmp_title.append("")
    if movieDatabase["Directed by"][i] == movieDatabase["Directed by"][i]:
        tmp_director.append(cleanQ(movieDatabase["Directed by"][i]))
    else:
        tmp_director.append("")
    if movieDatabase["Produced by"][i] == movieDatabase["Produced by"][i]:
        tmp_producer.append(cleanQ(movieDatabase["Produced by"][i]))
    else:
        tmp_producer.append("")
    if movieDatabase["Written by"][i] == movieDatabase["Written by"][i]:
        tmp_writer.append(cleanQ(movieDatabase["Written by"][i]))
    else:
        tmp_writer.append("")
    if movieDatabase["Starring"][i] == movieDatabase["Starring"][i]:
        tmp_starring.append(cleanQ(movieDatabase["Starring"][i]))
    else:
        tmp_starring.append("")
    if movieDatabase["Music by"][i] == movieDatabase["Music by"][i]:
        tmp_music.append(cleanQ(movieDatabase["Music by"][i]))
    else:
        tmp_music.append("")
    if movieDatabase["Release date"][i] == movieDatabase["Release date"][i]:
        tmp_date.append(cleanQ(str(movieDatabase["Release date"][i])))
    else:
        tmp_date.append("")
    if movieDatabase["Country"][i] == movieDatabase["Country"][i]:
        tmp_country.append(cleanQ(movieDatabase["Country"][i]))
    else:
        tmp_country.append("")
    if movieDatabase["Language"][i] == movieDatabase["Language"][i]:
        tmp_language.append(cleanQ(movieDatabase["Language"][i]))
    else:
        tmp_language.append("")

#  We are going to find all the words in the documents... Merge them together... And for each row sum them up and make the vocabulary


In [57]:
words=[]  # We are going to use this for creating the vocabulary.
clean_all = [] # This will be a new column with all the preprocessed words of the movie
for i in range(len(movieDatabase)):
    c_all = []
    for wordPl in tmp_plot[i]:
        words.append(wordPl)
        c_all.append(wordPl)
    for wordI in tmp_intro[i]:
        words.append(wordI)
        c_all.append(wordI)
    for wordT in tmp_title[i]:
        words.append(wordT)
        c_all.append(wordT)
    for wordD in tmp_director[i]:
        words.append(wordD)
        c_all.append(wordD)
    for wordPr in tmp_producer[i]:
        words.append(wordPr)
        c_all.append(wordPr)
    for wordW in tmp_writer[i]:
        words.append(wordW)
        c_all.append(wordW)
    for wordM in tmp_music[i]:
        words.append(wordM)
        c_all.append(wordM)
    for wordC in tmp_country[i]:
        words.append(wordC)
        c_all.append(wordC)
    for wordL in tmp_language[i]:
        words.append(wordL)
        c_all.append(wordL)
    clean_all.append(c_all)
movieDatabase["Clean All"] = pd.Series(clean_all)
movieDatabase.head(3)

/Users/salwaali/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,Title,Intro,Plot,Directed by,Produced by,Written by,Starring,Music by,Release date,Running time,Country,Language,Budget,URL,Clean All
0,Love by the Light of the Moon,Love by the Light of the Moon is a 1901 film b...,"The moon, painted with a smiling face hangs ov...",Edwin S. Porter,NaN,NaN,NaN,NaN,1901 (1901),NaN,United States,Silent,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"[moon, paint, smile, face, hang, park, night, ..."
1,The Martyred Presidents,The Martyred Presidents is a 1901 American fil...,"The film, just over a minute long, is composed...",Edwin S. Porter,NaN,NaN,See below,NaN,1901,1 minutes,United States,English,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"[film, minut, long, compos, two, shot, first, ..."
2,"Terrible Teddy, the Grizzly King","Terrible Teddy, the Grizzly King is a 1901 Ame...",Lasting just 61 seconds and consisting of two ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...","[last, 61, second, consist, two, shot, first, ..."


# Now we create the vocabulary. In the vocalulary there are all the words, and they are accompagnied by an unique index
# We are replacing the words in the column Clean All with the correponding index that it has on the vocabulary we've created



In [60]:
words=set(words)
words = list(words)
vocab={}
for i in range(len(words)):
    vocab.update({words[i] : i })
with open("vocabulary.json", "w", encoding = "utf8") as v:
    v.write(json.dumps(vocab))
    
# Replacing the words with it's ID in DF

for i in range(len(movieDatabase)):
    for j in range(len(movieDatabase["Clean All"][i])):
        if movieDatabase["Clean All"][i][j] in vocab.keys():
            movieDatabase["Clean All"][i][j] = vocab[movieDatabase["Clean All"][i][j]]

# We are going to change the new col and replace it with a counter dictionary, where every key is a word and the values are the number of times the word apears on the document 


In [67]:
from collections import Counter
movieDatabase["Clean All"] = movieDatabase["Clean All"].apply(lambda x : Counter(x))

/Users/salwaali/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



# Computing the TF 

In [68]:
def computeTF(doc_words):
    bow = 0
    for k, v in doc_words.items():
        bow = bow + v
    tf_word = {}
    for word, count in doc_words.items():
        tf_word[word] = count / float(bow)
    return tf_word

In [69]:
TF = []
for i in range(len(movieDatabase["Clean All"])):
    TF.append(computeTF(movieDatabase["Clean All"][i]))
movieDatabase["TF"] = TF

/Users/salwaali/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [70]:
movieDatabase.head(3)

,Title,Intro,Plot,Directed by,Produced by,Written by,Starring,Music by,Release date,Running time,Country,Language,Budget,URL,Clean All,TF
0,Love by the Light of the Moon,Love by the Light of the Moon is a 1901 film b...,"The moon, painted with a smiling face hangs ov...",Edwin S. Porter,NaN,NaN,NaN,NaN,1901 (1901),NaN,United States,Silent,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"{38525: 9, 91446: 1, 32484: 3, 61054: 1, 74249...","{38525: 0.10843373493975904, 91446: 0.01204819..."
1,The Martyred Presidents,The Martyred Presidents is a 1901 American fil...,"The film, just over a minute long, is composed...",Edwin S. Porter,NaN,NaN,See below,NaN,1901,1 minutes,United States,English,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"{69768: 2, 26974: 1, 5376: 2, 100939: 1, 84397...","{69768: 0.034482758620689655, 26974: 0.0172413..."
2,"Terrible Teddy, the Grizzly King","Terrible Teddy, the Grizzly King is a 1901 Ame...",Lasting just 61 seconds and consisting of two ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...","{107333: 1, 81865: 1, 35021: 2, 104220: 1, 843...","{107333: 0.008064516129032258, 81865: 0.008064..."


 # 2.1.1) Create your index!

In [62]:
# This dictionary has keys the term ids and as corresponding values the list of documents where the term id exists
# We save this dictionrary into a JSON file for future use

inverted_index = defaultdict(list)
for i in range(len(movieDatabase)):
    for keys, values in movieDatabase["Clean All"][i].items():
        inverted_index[keys].append(i)
        
# We save this dictionrary into a JSON file for future use        
with open("inverted_index.json", "w", encoding = "utf8") as i_d:
    i_d.write(json.dumps(inverted_index))


 # 2.2.1) Inverted Index


In [71]:
# We are creating a function to compute the tf_idf of a specific documents and a specific term
# We save this dictionrary into a JSON file for future use
idf = {}
for k, v in vocab.items():
    idf[v] = math.log(len(movieDatabase["Clean All"])/len(inverted_index[v]), 10)
with open("idf.json", "w", encoding = "utf8") as i_d_f:
    i_d_f.write(json.dumps(idf))

# We are computing the norm of the tf_idfs for all the documents and terms that exist on those
# movieDatabase["Clean All"] has all the words that exists in the document with the corresponding frequencies
# we store this information in a new column   
the_norm = []
for i in range(len(movieDatabase)):
    suma = 0
    for word in movieDatabase["Clean All"][i].keys():
        suma = suma + (movieDatabase["TF"][i][word]*idf[word])**2
    the_norm.append(suma**(1/2))
movieDatabase["Norm"] = the_norm

/Users/salwaali/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



# We are computing the norm of the tf_idfs for all the documents and terms that exist on those
# movieDatabase["Clean All"] has all the words that exists in the document with the corresponding frequencies
# we store this information in a new column  

In [73]:
the_norm = []
for i in range(len(movieDatabase)):
    suma = 0
    for word in movieDatabase["Clean All"][i].keys():
        suma = suma + (movieDatabase["TF"][i][word]*idf[word])**2
    the_norm.append(suma**(1/2))
movieDatabase["Norm"] = the_norm

# Calculated idf for all the terms in all the documents

In [74]:
# We are creating idf for all the terms in all the documents and we save it for future use
# We will need it for calculating the tf_idf for the query

idf = {}
for k, v in vocab.items():
    idf[v] = math.log(len(movieDatabase["Clean All"])/len(inverted_index[v]))
with open("idf.json", "w", encoding = "utf8") as i_d_f:
    i_d_f.write(json.dumps(idf))



# We create a function tf_idf so we can calculate the tfidf
# INPUT: docid, termid
# OUTPUT: tfidf for the input  

def tf_idf(docid, termid):
    return((movieDatabase["Clean All"][docid][termid]*idf[termid])/sum(movieDatabase["Clean All"][docid]))


# We create a dictionary inverted_index_2.

In [76]:
# We create a dictionary inverted_index_2. This dictionary has keys the term ids and as corresponding values the list of tuples.
# These tuples contain the documents where the term id exists and the corresponding NORMALIZED tf_idf

inverted_index_2 = defaultdict(list)
for k, v in inverted_index.items():
    for l in v:
        inverted_index_2[k].append((l, tf_idf(l,k)/movieDatabase["Norm"][l]))
        
with open("inverted_index_2.json", "w", encoding = "utf8") as i_d_2:
    i_d_2.write(json.dumps(inverted_index_2))


# 2.1.2) Execute the query

In [77]:
def SearchEngine1():
    movieDatabase = pd.read_csv(r"/Users/salwaali/Desktop/FINAL_ADM_HW3/ADM-HW3-master/searchenginedata1.csv", header=0)
    query = input()
    query=cleanQ(query)
    vocab=json.loads(open("vocabulary.json").read())
    inverted_index = json.loads(open("inverted_index.json").read())
    the_title = []
    the_intro = []
    the_url = []
    movies = range(len(movieDatabase))
    movies = set(movies)
    flag = 0
    for word in query:
        if word in vocab.keys():
            term_id = vocab[word]
            #print(term_id)
            movies = movies.intersection(inverted_index[str(term_id)])
            #print(movies)
        else:
            flag = 1
            return("Not Found")
            break
    if flag == 0:
        for index in movies:
            the_title.append(movieDatabase.Title[index])
            the_intro.append(movieDatabase.Intro[index])
            the_url.append(movieDatabase["URL"][index])
            
        fig = go.Figure(data=[go.Table(header=dict(values=['Title', 'Intro', 'URL']),
                 cells=dict(values=[the_title, the_intro, the_url]))
                     ])
        fig.update_layout(width=2000, height=1000)
        fig.show()

In [87]:
SearchEngine1()

zorba the greek cypriot


# Second search engin

In [98]:
def SearchEngine2(data = pd.read_csv(r"/Users/salwaali/Desktop/FINAL_ADM_HW3/ADM-HW3-master/searchenginedata1.csv", header=0)):
    query = input()
    if query == "":
        return("Get Out")
    query=cleanQ(query)
    the_similarity=[]
    the_title = []
    the_intro = []
    the_url = []
    movies = list(data.index)
    movies = set(movies)
    # ==================================================================================
    # I_M_P_O_R_T_I_N_G__A_L_L__T_H_E__N_E_C_E_S_S_A_R_Y__S_T_R_U_C_T_U_R_E_S__W_E__N_E_E_D
    # We need the words and their corresponding index for converting the query into the index
    vocab=json.loads(open("vocabulary.json").read())
    # We need the list of documents that have a specific word on it 
    inverted_index = json.loads(open("inverted_index.json").read())
    # We need the tf_idf of the doc i in respect to the term j
    _2nd_inverted_index=json.loads(open("inverted_index_2.json").read())
    # We need as output the title, into and url from the movieDatabase that we've created                               
    #movieDatabase = pd.read_csv(r"C:\Users\HP\Desktop\ADM HW3\searchenginedata1.csv", header=0)
    IDF = json.loads(open("idf.json").read())
    #data = pd.read_csv(r"C:\Users\HP\Desktop\ADM HW3\searchenginedata1.csv", header=0)
    #============================
    def TF(doc_words):
        bow = 0
        for k, v in doc_words.items():
            bow = bow + v
        tf_word = {}
        for word, count in doc_words.items():
            tf_word[word] = count / float(bow)
        return tf_word
    # ========================================================
    # C_R_E_A_T_E__T_H_E__V_E_C_T_O_R__F_O_R__T_H_E__Q_U_E_R_Y
    # We are replacing the string with its corresponding integer
    for word in query:
        if word in vocab.keys():
            query[query.index(word)]=(vocab[word])
        else:
            print("word Not found")
    query_vec = {}
    # We are creating a vector that eack component corresponds to the tf_idf for a specific term of the query     
    for word_ in query:
        query_vec[word_]=  TF(Counter(query))[word_]*IDF[str(word_)]
    ##=============================== 
    # C_R_E_A_T_E__T_H_E__V_E_C_T_O_R__F_O_R__E_A_C_H__D_O_C_U_M_E_N_T
    # We are checking which movies have all the terms of the query and we keep only them 
    for word in query: 
        movies = movies.intersection(inverted_index[str(word)])
    data = data.drop(index=data.index.difference(movies))
    doc_vec = {}
    for movie in movies:
        vec = []
        for word_ in query:
            index = _2nd_inverted_index[str(word_)]
            for i in range(len(index)):
                if index[i][0] == movie:
                    vec.append(index[i][1])
        doc_vec[movie] = vec
    def dot(vector_1, vector_2):
        sum = 0
        for i in range(len(vector_1)):
            sum = sum + vector_1[i]*vector_2[i]
        return(sum)
    def norm(vector):
        if len(vector) > 1:
            add = 0
            for i in range(len(vector)):
                add = add + vector[i]**2
            return add**(1/2)
        else:
            return(vector)
    similarity = {}
    for movie in movies:
        if isinstance(norm(list(query_vec.values())), float):
            similarity[movie] = dot(list(query_vec.values()),doc_vec[movie])/norm(list(query_vec.values()))
        else:
            similarity[movie] = dot(list(query_vec.values()),doc_vec[movie])/norm(list(query_vec.values()))[0]

    the_col = []
    for i in range(len(data)):
        for k, v in similarity.items():
            if data.index[i] == k:
                the_col.append(similarity[k])
    data["Similarity"] = the_col
    truth = []
    for movie_d in movies:
        if data["Similarity"][movie_d] in heapq.nlargest(5,data["Similarity"]):
            truth.append(movie_d)
    movieDatabase1 = data.drop(index=data.index.difference(truth))
    for index in truth:
            the_title.append(movieDatabase1.Title[index])
            the_intro.append(movieDatabase1.Intro[index])
            the_url.append(movieDatabase1["URL"][index])
            the_similarity.append(movieDatabase1["Similarity"][index])
    fig = go.Figure(data=[go.Table(header=dict(values=['Title', 'Intro', 'URL', 'Similarity']),
                 cells=dict(values=[the_title, the_intro, the_url, the_similarity]))
                     ])
    fig.update_layout(width=2000, height=1000)
    fig.show()
    return(movieDatabase1)
    

    

In [99]:
SearchEngine2()

love sex rock'n'roll


,Title,Intro,URL,Similarity
6334,The Girl Can't Help It,The Girl Can't Help It is a 1956 musical comed...,https://en.wikipedia.org/wiki/The_Girl_Can%27t...,0.000004
11377,Shout (film),Shout is a 1991 American musical romance film ...,https://en.wikipedia.org/wiki/Shout_(film),0.000014
15896,Rock of Ages (2012 film),Rock of Ages is a 2012 American musical comedy...,https://en.wikipedia.org/wiki/Rock_of_Ages_(20...,0.000009


# Third Search Engine

In [102]:
def SearchEngine3():
    movie_first = SearchEngine2()
    if isinstance(movie_first, str):
        return("Sorry the search was unsuccessful")
    while True :
        movie_last = movie_first
        print("Refine your search. If you don't want to, press enter.")
        movie_first = SearchEngine2(data = movie_last)
        if isinstance(movie_first, str):
            break
    return(movie_last)
    

In [103]:
SearchEngine3()

love sex rock'n'roll


Refine your search. If you don't want to, press enter.
jukebox


Refine your search. If you don't want to, press enter.
jayne


Refine your search. If you don't want to, press enter.



,Title,Intro,URL,Similarity
6334,The Girl Can't Help It,The Girl Can't Help It is a 1956 musical comed...,https://en.wikipedia.org/wiki/The_Girl_Can%27t...,0.000005


In [105]:
SearchEngine3()

stanley kubrick


Refine your search. If you don't want to, press enter.
awards


Refine your search. If you don't want to, press enter.



,Title,Intro,URL,Similarity
13375,A.I. Artificial Intelligence,"A.I. Artificial Intelligence, also known as A....",https://en.wikipedia.org/wiki/A.I._Artificial_...,0.000001


# You've seen our searches. Now its your turn. Choose the search engine you want and ennjoy the results...

In [109]:
search_engine = input("Pick your Search Engine 1 , 2 or 3 ? ")

if search_engine == "1":
    SearchEngine1()
elif search_engine == "2":
    SearchEngine2()
elif search_engine == "3":
    SearchEngine3()
    
else:
    print("The Search Engine you chose doesn’t exist")

Pick your Search Engine 1 , 2 or 3 ? 2
robert sheehan
